#### Explaining WASM

In [ ]:
def runwasm(wasmfile):
    from IPython.display import display, Javascript
    display(Javascript("""
    const params = {
        P0lib: {
            write: i => element.append(i + ' '),
            writeln: () => element.append(document.createElement('br')),
            read: () => window.prompt()
        }
    }
    
    var wasmByteString = \"""" + str(open(wasmfile, "rb").read()) + """\"; // pass the wasm file to JavaScript as byte string
    wasmByteString = wasmByteString.substring(2, wasmByteString.length - 1); // remove the byte literals b'...'
    const wasmArrayBuffer = new Uint8Array(wasmByteString.length); // convert the binary string to ArrayBuffer
    for (let i = 0; i < wasmByteString.length; i++)
      wasmArrayBuffer[i] = wasmByteString.charCodeAt(i);
    
    WebAssembly.compile(wasmArrayBuffer.buffer) // compile (sharable) code
        .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
        // .then(instance => instance.exports.program()); // run the main program; not needed if a start function is specified
     """))

In [ ]:
def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i, end=' ')
    def writeln(s): print()
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

In [ ]:
from wasmer import engine, Store, Module, Instance, ImportObject, Function
from wasmer_compiler_cranelift import Compiler

def runwasmer(wasmfile):
    def write(i: int): print(i, end=' ')
    def writeln(): print()
    def read() -> int: return int(input()) 
    store = Store(engine.JIT(Compiler))
    module = Module(store, open(wasmfile, 'rb').read())
    import_object = ImportObject()
    import_object.register("P0lib", {"write": Function(store, write),
                                     "writeln": Function(store, writeln),"read": Function(store, read)})
    instance = Instance(module, import_object)

Consider the following WebAssembly program:

In [ ]:
%%writefile add.wat
(module
  (import "P0lib" "write" (func $write (param i32)))
  (global $x (mut i32) i32.const 3)
  (global $y (mut i32) i32.const 4)
  (func $program
    global.get $x
    global.get $y
    i32.add
    call $write
  )
  (start $program)
)

Explain what each line is for!

YOUR ANSWER HERE

Instructor's Answer:

```
(module
  (import "P0lib" "write" (func $write (param i32)))
```

Makes procedure `write` from the module `P0lib`, which has to be provided by the host environment, available under the name `$write`; it takes one `i32` parameter.

```
  (global $x (mut i32) i32.const 3)
  (global $y (mut i32) i32.const 4)
```

Declares mutable global variables `$x` and `$y` of type `mut i32`; the instruction `i32.const  3` initializes `x` by pushing `3` on the stack (all global variables have to be initialized).

```
  (func $program
```

Declares procedure `$program`.
```
    global.get $x
    global.get $y
```

Takes global variables `$x` and `$y` and pushes them on the stack.

```
    i32.add
```

Pops the top two integers from the stack and pushes their sum back on the stack.

```
    call $write
  )
```

Pops the top element from the stack and passes it to the imported `$write` procedure.

```
  (start $program)
)
```

Declares `$program` to be the main program, i.e. the procedure that is called by the host environment.

The text file `add.wat` is first converted to binary file `add.wasm`:

In [ ]:
!wat2wasm add.wat

This file can now be run in the browser by a just-in-time compiler, with JavaScript as the host environment:

In [ ]:
runwasm("add.wasm")

Alternatively, it can be interpreted in Python, with Python as the host environment:

In [ ]:
runpywasm("add.wasm")

It can also be run by an external compiler, but still use Python as the host environment:

In [ ]:
runwasmer("add.wasm")